# FMLIKH Model: Large Amplitude Oscillatory Shear (LAOS)

## Protocol-Specific Context (Multi-Mode)

**FMLIKH LAOS** combines **multi-mode yielding** with **fractional memory** under large-amplitude oscillation:

1. **Intra-cycle multi-mode evolution**: Each $\lambda_i(t)$ changes within cycles via $D_t^{\alpha_i} \lambda_i$
2. **Distributed yielding**: Fast modes yield early in cycle, slow modes resist longer
3. **Rich harmonic content**: Each mode contributes harmonics with power-law decay $I_{n,i} \sim n^{-\alpha_i}$

**Why this matters**: Single-mode LAOS shows one yielding event per cycle. FMLIKH with N=3 can show **multiple intra-cycle yielding events** as successive structural levels break under oscillatory strain. This produces complex Lissajous figures and harmonic spectra that match experimental LAOS of hierarchical materials.

> **Physical insight**: During LAOS, fast modes break/reform rapidly within each cycle (high-frequency response). Slow modes persist across cycles (low-frequency envelope). Fractional kinetics smooths transitions, producing distinct harmonic signatures.

> **Handbook:** See [FMLIKH Documentation](../../docs/source/models/fikh/fmlikh.rst) for multi-mode LAOS harmonics and Lissajous interpretation.

## Learning Objectives

1. Simulate FMLIKH LAOS with N=3 modes
2. Analyze Lissajous figures showing multi-mode yielding
3. Extract harmonics and observe mode-specific power-law decay
4. Compare FMLIKH vs single-mode FIKH LAOS response
5. Understand intra-cycle structure evolution for each mode

## Prerequisites

- NB07: FMLIKH flow curve
- NB06: FIKH LAOS (single-mode baseline)

**Estimated Time:** 5-8 minutes (fast), 20-30 minutes (full)

## 1. Setup

In [ ]:
import sys
IN_COLAB = "google.colab" in sys.modules
if IN_COLAB:
    %pip install -q rheojax
    import os
    os.environ["JAX_ENABLE_X64"] = "true"

In [ ]:
%matplotlib inline
import os, sys, time, warnings
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display

from rheojax.core.jax_config import safe_import_jax, verify_float64
from rheojax.models.fikh import FMLIKH

# Robust path resolution for execution from any directory
from pathlib import Path
_nb_dir = Path(__file__).parent if "__file__" in dir() else Path.cwd()
_utils_candidates = [_nb_dir / ".." / "utils", Path("examples/utils"), _nb_dir.parent / "utils"]
for _p in _utils_candidates:
    if (_p / "fikh_tutorial_utils.py").exists():
        sys.path.insert(0, str(_p.resolve()))
        break
from fikh_tutorial_utils import (
    load_pnas_laos, save_fikh_results, print_convergence_summary,
    compute_fit_quality, get_fmlikh_param_names,
)

# Shared plotting utilities
sys.path.insert(0, os.path.dirname(os.path.abspath("")))
from utils.plotting_utils import (
    plot_nlsq_fit,
    display_arviz_diagnostics,
    plot_posterior_predictive,
)

jax, jnp = safe_import_jax()
verify_float64()

# ============================================================
# FAST_MODE Configuration
# ============================================================
FAST_MODE = os.environ.get("FAST_MODE", "1") == "1"

if FAST_MODE:
    print("Running in FAST_MODE - reduced samples for quick validation")
    DOWNSAMPLE_FACTOR = 4  # Use every 4th data point
    NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 100, 200, 1
else:
    print("Running in FULL mode - complete Bayesian inference")
    DOWNSAMPLE_FACTOR = 1  # Use all data
    NUM_WARMUP, NUM_SAMPLES, NUM_CHAINS = 200, 500, 1

## 2. Load Data

In [ ]:
OMEGA = 1.0
STRAIN_AMP_INDEX = 5

time_raw, strain_raw, stress_raw = load_pnas_laos(omega=OMEGA, strain_amplitude_index=STRAIN_AMP_INDEX)

# Apply downsampling for FAST_MODE
if DOWNSAMPLE_FACTOR > 1:
    time_data = time_raw[::DOWNSAMPLE_FACTOR]
    strain_data = strain_raw[::DOWNSAMPLE_FACTOR]
    stress_data = stress_raw[::DOWNSAMPLE_FACTOR]
    print(f"Downsampled {len(time_raw)} -> {len(time_data)} points (factor {DOWNSAMPLE_FACTOR}x)")
else:
    time_data, strain_data, stress_data = time_raw, strain_raw, stress_raw

gamma_0 = (np.max(strain_data) - np.min(strain_data)) / 2

print(f"Data points: {len(time_data)}")
print(f"Strain amplitude: {gamma_0:.4f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(strain_data, stress_data, "k-", lw=1)
ax1.set_xlabel("Strain [-]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title("Elastic Lissajous")
ax1.grid(True, alpha=0.3)

gamma_dot_data = np.gradient(strain_data, time_data)
ax2.plot(gamma_dot_data, stress_data, "k-", lw=1)
ax2.set_xlabel("Strain rate [1/s]")
ax2.set_ylabel("Stress [Pa]")
ax2.set_title("Viscous Lissajous")
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 3. NLSQ Fitting

In [ ]:
# Number of modes: reduced in FAST_MODE for speed
N_MODES = 2 if FAST_MODE else 3
print(f"Using {N_MODES} modes (FAST_MODE={FAST_MODE})")

model = FMLIKH(n_modes=N_MODES, include_thermal=False, shared_alpha=True, alpha_structure=0.7)

t0 = time.time()
model.fit(time_data, stress_data, test_mode="laos", strain=strain_data, method='scipy')
print(f"NLSQ time: {time.time() - t0:.2f} s")

# Predict
laos_result = model.predict(time_data, test_mode="laos", strain=strain_data)
stress_pred = np.asarray(laos_result)
metrics = compute_fit_quality(stress_data, stress_pred)
print(f"R² = {metrics['R2']:.6f}")

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(strain_data, stress_data, "ko", markersize=2, alpha=0.5, label="Data")
ax1.plot(strain_data, stress_pred, "-", lw=2, color="C0", label="FMLIKH fit")
ax1.set_xlabel("Strain [-]")
ax1.set_ylabel("Stress [Pa]")
ax1.set_title(f"Elastic Lissajous (R² = {metrics['R2']:.4f})")
ax1.legend()
ax1.grid(True, alpha=0.3)

ax2.plot(time_data, stress_data, "ko", markersize=2, alpha=0.5, label="Data")
ax2.plot(time_data, stress_pred, "-", lw=2, color="C0", label="FMLIKH fit")
ax2.set_xlabel("Time [s]")
ax2.set_ylabel("Stress [Pa]")
ax2.set_title("Stress Time Series")
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
display(fig)
plt.close(fig)

## 4. Bayesian Inference

In [ ]:
param_names = get_fmlikh_param_names(n_modes=N_MODES, shared_alpha=True)
initial_values = {n: model.parameters.get_value(n) for n in param_names if n in model.parameters}

print(f"Running NUTS: {NUM_WARMUP} warmup + {NUM_SAMPLES} samples x {NUM_CHAINS} chain(s)")
print(f"  FAST_MODE={FAST_MODE}, N_MODES={N_MODES}, data points={len(time_data)}")

t0 = time.time()
result = model.fit_bayesian(
    time_data, stress_data, test_mode="laos", strain=strain_data,
    num_warmup=NUM_WARMUP, num_samples=NUM_SAMPLES, num_chains=NUM_CHAINS,
    initial_values=initial_values, seed=42,
)
t_bayes = time.time() - t0
print(f"Bayesian time: {t_bayes:.1f} s")

In [ ]:
key_params = ["G_0", "G_1", "alpha_structure", "sigma_y0"]
print_convergence_summary(result, key_params)

print("\n### Diagnostic Interpretation")
print("| Metric | Target | Meaning |")
print("|--------|--------|---------|")
print("| **R-hat** | < 1.01 | Chain convergence |")
print("| **ESS** | > 400 | Sample quality |")
print("| **Divergences** | < 1% | NUTS reliability |")

display_arviz_diagnostics(result, key_params, fast_mode=FAST_MODE)

### Convergence Diagnostics

**Diagnostic Targets:**

| Metric | Target | Meaning |
|--------|--------|---------|
| **R-hat** | < 1.01 | Chain convergence |
| **ESS** | > 400 | Sample quality |
| **Divergences** | < 1% | NUTS reliability |

## 5. Save Results

In [ ]:
save_fikh_results(model, result, "fmlikh", "laos", param_names)

## 5. Save Results

## Key Takeaways

1. **FMLIKH LAOS: multi-mode yielding under oscillation**
2. **Distributed intra-cycle structure evolution** from each mode
3. **Complex Lissajous figures** with multiple yielding features
4. **Harmonics decay as $I_{n,i} \sim n^{-\alpha_i}$ per mode**
5. **Hierarchical structure produces rich nonlinear response**
6. **Residual analysis** of harmonics validates power-law decay per mode

---

## Further Reading

### RheoJAX Handbook

- **[FMLIKH Complete Guide](../../docs/source/models/fikh/fmlikh.rst)**: Multi-mode equations, mode selection, parameter estimation
- **[FIKH Model Family Overview](../../docs/source/models/fikh/index.rst)**: When to use FIKH vs FMLIKH, material recommendations
- **[Fractional Calculus in Rheology](../../docs/source/models/fikh/fikh.rst#theoretical-background)**: Caputo derivatives and Mittag-Leffler functions

### Key References

**Fractional Multi-Mode Theory:**
1. Wei, Y., Solomon, M.J., & Larson, R.G. (2018). "A multimode structural kinetics constitutive equation." *J. Rheol.*, 62(1), 321-342.
2. Jaishankar, A. & McKinley, G.H. (2014). "A fractional K-BKZ constitutive formulation." *J. Rheol.*, 58, 1751-1788.

**Fractional Calculus:**
3. Podlubny, I. (1999). *Fractional Differential Equations*. Academic Press.
4. Mainardi, F. (2010). *Fractional Calculus and Waves in Linear Viscoelasticity*. Imperial College Press.

**IKH Foundation:**
5. Dimitriou, C.J. & McKinley, G.H. (2014). "A comprehensive constitutive law for waxy crude oil." *Soft Matter*, 10, 6619-6644.
6. Geri, M. et al. (2017). "Thermokinematic memory and the thixotropic elasto-viscoplasticity of waxy crude oils." *J. Rheol.*, 61(3), 427-454.

---

### Next Steps

You've completed all **12 FIKH/FMLIKH tutorials**:

**FIKH (Single-Mode, NB01-06):**
- Flow curve, startup, relaxation, creep, SAOS, LAOS
- **Key signature**: Power-law memory via fractional derivative $D_t^{\alpha}$

**FMLIKH (Multi-Mode, NB07-12):**
- Same protocols with N=3 modes
- **Key signature**: Hierarchical relaxation + fractional memory per mode

### Applications

- **Waxy crude oils**: Pipeline restart, thermal cycling
- **Colloidal gels**: Hierarchical aggregation, aging
- **Drilling fluids**: Gel strength evolution, borehole stability
- **Food gels**: Texture recovery, processing windows
- **Greases**: Cold-start torque, shear stability